In [31]:
# https://github.com/TomSchimansky/TkinterMapView

In [32]:
import tweepy
from tkinter import *
import tkinter as tk
import tkintermapview as tkm
from geopy.geocoders import Nominatim
import pickle
import math

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [34]:
# [13:51, 11/03/2023] Abhay: api key qkBxRRYAv1KGXLLQqg6FkzlP4
# [13:52, 11/03/2023] Abhay: api key secret 3rAK403gUYnKAmccEO2sQOKjnakjZuPS9uyQZ97UZuSrnalAUi
# [13:52, 11/03/2023] Abhay: bearer token AAAAAAAAAAAAAAAAAAAAAE2FhwEAAAAAlknGU863CmXhK7OCQLNZxnQtEZ0%3DE1OsLHbSOtLvPG4VQlWWO7WlOkJpxnzmMkRig0AuxIKIDQ1UGY
# [13:53, 11/03/2023] Abhay: access token 1574425806545117184-7A5qOM7dRe5UjCCuUVdzjbmuhOeICr
# [13:53, 11/03/2023] Abhay: access token secret lyz9sy828kUYX0c2429lce4eCjFJR0RHKx0189BczBczz

In [35]:
#fetch live tweets
import tweepy

# Variables that contains the credentials to access Twitter API
ACCESS_TOKEN = '1574425806545117184-7A5qOM7dRe5UjCCuUVdzjbmuhOeICr'
ACCESS_SECRET = 'lyz9sy828kUYX0c2429lce4eCjFJR0RHKx0189BczBczz'
CONSUMER_KEY = 'qkBxRRYAv1KGXLLQqg6FkzlP4'
CONSUMER_SECRET = '3rAK403gUYnKAmccEO2sQOKjnakjZuPS9uyQZ97UZuSrnalAUi'


# Setup access to API
def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    api = tweepy.API(auth)
    return api


# Create API object
api = connect_to_twitter_OAuth()

In [36]:
root_tk = tk.Tk()
root_tk.geometry(f"{800}x{600}")
root_tk.title("map_view.py")



''

In [37]:
model_ops_list = ["BERT", "SVC"]
value_inside = tk.StringVar(root_tk)# text in the select option
value_inside.set("Select a Model")
model_select = tk.StringVar(root_tk)# text in the select option
drop = tk.OptionMenu(root_tk, value_inside, *model_ops_list,
#                      relief = GROOVE
                    )
drop.grid(row=0, column=0)


In [38]:
def search_event(event=None):
    map_widget.set_address(search_bar.get())

In [39]:
search_bar = tk.Entry(root_tk, width=50, relief = GROOVE)
search_bar.grid(row=0, column=1, pady=10, padx=10, sticky="we")
search_bar.focus()

search_bar_button = tk.Button(root_tk, width=8, text="Search", command=search_event, relief = GROOVE)
search_bar_button.grid(row=0, column=2, pady=10, padx=10)

# search_bar_clear = tk.Button(root_tk, width=8, text="Clear", command=clear)
# search_bar_clear.grid(row=0, column=2, pady=10, padx=10)

In [40]:
# create map widget
map_widget = tkm.TkinterMapView(root_tk, width=800, height=600, corner_radius=0)
map_widget.place(relx=0.5, rely=0.5, anchor=tk.CENTER)
map_widget.grid(row=1, column=1, columnspan=2, sticky="nsew")
# tk.search_marker = None
# tk.search_in_progress = False

In [41]:
def define_poly(coords):
    deg = 0
    poly = []
    while deg < 2*math.pi:# in the form [()]
        poly.append((coords[0] + (0.01*math.cos(deg)), coords[1] + (0.01*math.sin(deg))))
        deg = deg + (2*math.pi/360)
    return poly

In [42]:
def add_marker_event(coords):
#     global to_be_classified
    print("Add marker:", coords)
    new_marker = map_widget.set_marker(coords[0], coords[1], text="new marker")
    poly = define_poly(coords)
#     print(new_marker.position, new_marker.text)  # get position and text
    to_be_classified = find_tweet(coords)
    pred_condition = prediction(to_be_classified)
    if pred_condition:
        polygon_1 = map_widget.set_polygon(poly, fill_color= "red", outline_color="red", name="disaster_affected_area")
    else:
        polygon_1 = map_widget.set_polygon(poly, fill_color= "green", outline_color="green", name="disaster_affected_area")
    
    print(f"\n\n {to_be_classified}")
    

    

map_widget.add_right_click_menu_command(label="Add Marker",
                                        command=add_marker_event,
                                        pass_coords=True)


In [43]:
# Initialize Nominatim API
def find_tweet(coords):
    to_be_classified = []#this is going to be passed ahead
    coord_str = str(coords[0]) + ", " + str(coords[1])     
    print("cordinates are: ", coords)
    geolocator = Nominatim(user_agent="MyApp")
    # Latitude & Longitude input
    coordinates = coord_str
    location = geolocator.reverse(coordinates)
    address = location.raw['address']
    # Traverse the data
    city = address.get('city', '')
    state = address.get('state', '')
    country = address.get('country', '')
    print(f"{city} \n{state} \n{country} \n")
    places = api.search_geo(lat = coords[0], long = coords[1], granularity="city")
    # for place in places:    
    #     print("placeid:%s" % place)
    placed = "placeid:%s" % places
    #item iterator class 
    public_tweets = tweepy.Cursor(api.search_tweets, count=5,q="place:%s" % places[0].id, tweet_mode="extended", lang = "en").items(50)                            
    for tweet in public_tweets:
        to_be_classified.append(tweet.full_text)
        print("result of the find tweet function", tweet.full_text)
    return to_be_classified

In [44]:
def prediction(to_be_classified):
    print(value_inside.get())
    if value_inside.get() == "SVC":
        filename = '/Major_Project/models/text_clf.sav'
        loaded_model = pickle.load(open(filename, 'rb'))
    elif value_inside.get() == "BERT":
        import os
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
        import tensorflow as tf
        import tensorflow_hub as hub
        import tensorflow_text as text
        from sklearn.model_selection import train_test_split
        print('Tensorflow module imported')
        loaded_model = tf.keras.models.load_model('/Major_Project/models/bert_success_1/bert_success_1')
    else:
        filename = '/Major_Project/models/text_clf.sav'
        loaded_model = pickle.load(open(filename, 'rb'))
        
    predictions = loaded_model.predict(to_be_classified)
    print(f"\n\n {loaded_model.predict(to_be_classified)}")
    disaster = 0 #count number of disaster events
    for i in predictions:
        if i:
            disaster +=1
    dis_cond = disaster / len(predictions)
    if dis_cond > 0.5:
        return 1
    else:
        return 0


In [ ]:
pip install

In [45]:
root_tk.mainloop()

Add marker: (52.516594443015485, 13.37689033729552)
cordinates are:  (52.516594443015485, 13.37689033729552)
Berlin 
 
Deutschland 

result of the find tweet function @slowberlin Sitting in here now, reading this tweet.
result of the find tweet function Excited to get to #FlutterForward Berlin event 💙  
Despite all the odds!! https://t.co/9ykvOH6Uys
result of the find tweet function @timthetwinky Sexy
result of the find tweet function Iheanacho to Watkins for a - 4? With him being out of form and Barnes being back as well as Daka starting the last game and scoring I’m concerned about his minutes! Thoughts? #FPL
result of the find tweet function Why this flyer rip off Smile like that? https://t.co/CcfEHrEf7W https://t.co/0hZ6oNRpOM
result of the find tweet function He couldn’t find elephants in Nairobi, but got a lot of insights about #flutter Now @ShuregDenisov is on the stage with “Flutter Forward Takeaways. What's next?” 💙 #flutterforward ✨ https://t.co/7oGdd1bA8l
result of the find 

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\abhi1\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\abhi1\AppData\Local\Temp\ipykernel_10420\1468744194.py", line 8, in add_marker_event
    pred_condition = prediction(to_be_classified)
  File "C:\Users\abhi1\AppData\Local\Temp\ipykernel_10420\177946943.py", line 11, in prediction
    import tensorflow_text as text
  File "C:\Users\abhi1\AppData\Roaming\Python\Python39\site-packages\tensorflow_text\__init__.py", line 21, in <module>
    from tensorflow_text.python import keras
  File "C:\Users\abhi1\AppData\Roaming\Python\Python39\site-packages\tensorflow_text\python\keras\__init__.py", line 21, in <module>
    from tensorflow_text.python.keras.layers import *
  File "C:\Users\abhi1\AppData\Roaming\Python\Python39\site-packages\tensorflow_text\python\keras\layers\__init__.py", line 22, in <module>
    from tensorflow_text.python.keras.layers.token